### Packages

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


### Loading the data

In [26]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "2010-01-01", end = "2021-06-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [27]:
df_comp = raw_data.copy()

In [28]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [29]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

### Creating Returns

In [30]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [31]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [33]:
from pmdarima.arima import auto_arima

In [32]:
model_auto = auto_arima(df.ret_ftse[1:])

In [34]:
model_auto

ARIMA(order=(2, 0, 2), scoring_args={}, with_intercept=False)

In [35]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 2389
Model:               SARIMAX(2, 0, 2)   Log Likelihood               -3210.436
Date:                Thu, 17 Jun 2021   AIC                           6430.873
Time:                        00:56:36   BIC                           6459.766
Sample:                             0   HQIC                          6441.387
                               - 2389                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0260      0.035      0.738      0.461      -0.043       0.095
ar.L2          0.8716      0.036     24.442      0.000       0.802       0.941
ma.L1         -0.0206      0.030     -0.680      0.497      -0.080       0.039
ma.L2         -0.9175      0.031    -29.769      0.000      -0.978      -0.857
sigma2         0.8605      0.016     52.307      0.000       0.828       0.893
===================================================================================
Ljung-Box (Q):                       37.56   Jarque-Bera (JB):               717.79
Prob(Q):                              0.58   Prob(JB):                         0.00
Heteroskedasticity (H):               0.49   Skew:                            -0.24
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [14]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [15]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                                 y   No. Observations:                 6276
Model:             SARIMAX(2, 0, 2)x(1, 0, 1, 5)   Log Likelihood               -6340.661
Date:                           Fri, 03 Jan 2020   AIC                          12705.322
Time:                                   10:34:50   BIC                          12786.256
Sample:                               01-10-1994   HQIC                         12733.364
                                    - 01-29-2018                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0043      0.009     -0.500      0.617      -0.021       0.013
drift       -1.45e-06   2.72e-06     -0.533      0.594   -6.78e-06    3.88e-06
ret_spx        0.0958      0.006     16.947      0.000       0.085       0.107
ret_dax        0.5582      0.005    113.729      0.000       0.549       0.568
ret_nikkei     0.0702      0.004     16.547      0.000       0.062       0.079
ar.L1         -0.1932      0.101     -1.918      0.055      -0.390       0.004
ar.L2          0.5083      0.059      8.643      0.000       0.393       0.624
ma.L1          0.0863      0.101      0.855      0.393      -0.112       0.284
ma.L2         -0.5369      0.056     -9.535      0.000      -0.647      -0.427
ar.S.L5        0.0705      0.339      0.208      0.836      -0.595       0.736
ma.S.L5       -0.0987      0.339     -0.291      0.771      -0.764       0.567
sigma2         0.4404      0.004     99.131      0.000       0.432       0.449
===================================================================================
Ljung-Box (Q):                       84.27   Jarque-Bera (JB):             15325.76
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.54   Skew:                             0.25
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""